In project, we'll use SQL in Jupyter Notebook to explore and analyze data

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Overview of the Data
We'll begin by getting a sense of what the data looks like.

In [2]:
%%sql
select *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


#### Here are the descriptions for some of the columns:
- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
Let's start by calculating some summary statistics and see what they tell us.

# Summary Statistics

In [3]:
%%sql
SELECT MIN(population) min_population,
    MAX(population) max_population,
    MIN(population_growth) min_population_growth,
    MAX(population_growth) max_population_growth
FROM facts;

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

### Country with minimum population

In [6]:
%%sql
SELECT *
FROM facts
WHERE population == (SELECT MIN(population) FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


 It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica:

# Country with maximum population

In [7]:
%%sql
SELECT * 
FROM facts
WHERE population == (SELECT MAX(population)
                     FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion.

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

In [8]:
%%sql
SELECT MIN(population) min_population,
    MAX(population) max_population,
    MIN(population_growth) min_population_growth,
    MAX(population_growth) max_population_growth
FROM facts
WHERE name NOT IN ("World");

Done.


min_population,max_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

## Exploring Average Population and Area
Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.

In [9]:
%%sql
SELECT AVG(population) AS avg_population,
    AVG(area) AS avg_area
    FROM facts
    WHERE name <> "World";

Done.


avg_population,avg_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

Finding Densely Populated Countries
To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

Above average values for population.
Below average values for area.

In [14]:
%%sql
SELECT name, population
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name <> "World")
AND area < (SELECT AVG(area) from facts WHERE name <> "World");

Done.


name,population
Bangladesh,168957745
Germany,80854408
Iraq,37056169
Italy,61855120
Japan,126919659
"Korea, South",49115196
Morocco,33322699
Philippines,100998376
Poland,38562189
Spain,48146134


Some of these countries are generally known to be densely populated, so we have confidence in our results!

### Countries with most people

In [17]:
%%sql
SELECT name, population
    FROM facts
    WHERE population == (SELECT MAX(population)
                         FROM facts
                         WHERE name <> "World");

Done.


name,population
China,1367485388


### Country with highest growth rate

In [18]:
%%sql
SELECT name, population_growth
    FROM facts
    WHERE population_growth == (SELECT MAX(population_growth)
                         FROM facts
                         WHERE name <> "World");

Done.


name,population_growth
South Sudan,4.02


### Country having highest water to land ratio

In [30]:
%%sql
SELECT name, CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT) water_land_ratio
    FROM facts
    WHERE CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT) == (SELECT MAX(CAST(area_water AS FLOAT) / CAST(area_land AS FLOAT)) water_land_ratio
    FROM facts
    WHERE name <> "World");

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666


### Countries that have more water than land

In [31]:
%%sql
SELECT name, area_water, area_land
    FROM facts
    WHERE area_water > area_land;

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


### Countries with high death rate than birth rate

In [35]:
%%sql
SELECT name, birth_rate, death_rate
    FROM facts
    WHERE birth_rate < death_rate;

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


In [34]:
%%sql
SELECT * FROM facts LIMIT 2;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3


### Countries having the highest population/area ratio

In [46]:
%%sql
SELECT name, CAST(population AS FLOAT) / CAST(area_land AS FLOAT) pop_area_ratio
FROM facts
ORDER BY pop_area_ratio DESC LIMIT 20;

Done.


name,pop_area_ratio
Macau,21168.964285714286
Monaco,15267.5
Singapore,8259.784570596797
Hong Kong,6655.27120223672
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259
